In [ ]:
import requests
from bs4 import BeautifulSoup

In [ ]:
with open("./data/game_urls_filtered.txt", "r") as f:
    urls = f.read().splitlines()

for url in urls:
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")